In [2]:
# import libraries
import pandas as pd
import numpy as np
import pickle

In [3]:
!pip install xlrd

  Using cached xlrd-2.0.1-py2.py3-none-any.whl (96 kB)


In [5]:
dictNonChaebol = {}

# 2001

In [38]:
year = '2001'
df = pd.read_excel("../data/kftc/affilNumTtlAsset/affilNumTtlAsset01.xls", skiprows=[0, 1, 2], header=[0, 1])
df.iloc[:, 1] = df.iloc[:, 1].ffill()
df = df[df.iloc[:, 1] != '지정 제외 집단']
# print(df.iloc[:, 1].unique())
df = df.copy().iloc[:, [2, 3, 4, 10]]
df.columns = ['chaebol', '동일인', 'n_affil_2001', 'ttl_asset_2001']
lstNonChaebol = ['포항제철', '현대정유', '하나로통신', '대우전자', '고합']
dictNonChaebol[year] = lstNonChaebol
df = df.drop([0, 16, 24, 25, 29])
df = df.sort_values(['ttl_asset_2001'], ascending=False)
df['rank'] = df['ttl_asset_2001'].rank(ascending=False)
df.to_excel("../data/overview/affilNumTtlAsset/affilNumTtlAsset2001.xlsx", index=False)
df.tail(50)

,chaebol,동일인,n_affil_2001,ttl_asset_2001,rank
6,삼성,이건희,64.0,69873.0,1.0
14,현대,정몽헌,26.0,53632.0,2.0
9,엘지,구본무,43.0,51965.0,3.0
8,에스케이,최태원,54.0,47379.0,4.0
28,현대자동차,정몽구,16.0,36136.0,5.0
12,한진,조중훈,19.0,21307.0,6.0
5,롯데,신격호,31.0,16694.0,7.0
1,금호,박성용,17.0,11606.0,8.0
13,한화,김승연,25.0,11496.0,9.0
19,두산,박용곤,18.0,11192.0,10.0


# Loop

In [36]:
dictNonChaebol = {}
for yr in range(2001, 2023):
    year = str(yr)
    print("="*20+year+"="*20)
    df = pd.read_excel(f"../data/raw/kftc/affilNumTtlAsset/affilNumTtlAsset{year[2:]}.xls", skiprows=[0, 1, 2], header=[0, 1])
    df.iloc[:, 1] = df.iloc[:, 1].ffill()
    df.iloc[:, 2] = df.iloc[:, 2].ffill()
    df = df[df.iloc[:, 1] != '지정 제외 집단']
    df = df[df.iloc[:, 2] != '계']
    # print(df.iloc[:, 1].unique())
    df = df.copy().iloc[:, [2, 3, 4, 10]]
    df.columns = ['chaebol', f'the_same_person_{year}', f'n_affil_{year}', f'ttl_asset_{year}']
    rowsToDrop = list(df[df[f'the_same_person_{year}'].str.len() >= 4].index)
    rowsToDrop.extend(df[df[f'the_same_person_{year}'] == '포스코'].index)
    lstNonChaebol = list(df[df[f'the_same_person_{year}'].str.len() >= 4][f'the_same_person_{year}'])
    for grp in lstNonChaebol:
        if ('포스코' in grp) or ('포항제철' in grp):
            break
    else:
        lstNonChaebol.append('포스코')
    print(lstNonChaebol)
    dictNonChaebol[year] = lstNonChaebol
    df = df.drop(rowsToDrop)
    df = df.sort_values([f'ttl_asset_{year}'], ascending=False)
    df[f'rank_{year}'] = df[f'ttl_asset_{year}'].rank(ascending=False, method='min')
    df.to_excel(f"../data/overview/affilNumTtlAsset/affilNumTtlAsset{year}.xlsx", index=False)
    
    df['chaebol'].replace('에스케이\(주\)', '에스케이', regex=True, inplace=True)
    df['chaebol'].replace('DB', '동부', regex=True, inplace=True)
    df['chaebol'].replace('DL', '대림', regex=True, inplace=True)
    df['chaebol'].replace('KCC', '케이씨씨', regex=True, inplace=True)
    if year == '2001':
        dfMrgd = df
    else:
        dfMrgd = dfMrgd.merge(df, how='outer', on='chaebol')
    print(df.head(50))
    print("="*20+year+"="*20)

====================2001====================
['(주)고합', '대우전자(주)', '포항제철', '하나로통신', '현대정유(주)']
   chaebol the_same_person_2001  n_affil_2001  ttl_asset_2001  rank_2001
6       삼성                  이건희          64.0         69873.0        1.0
14      현대                  정몽헌          26.0         53632.0        2.0
9       엘지                  구본무          43.0         51965.0        3.0
8     에스케이                  최태원          54.0         47379.0        4.0
28   현대자동차                  정몽구          16.0         36136.0        5.0
12      한진                  조중훈          19.0         21307.0        6.0
5       롯데                  신격호          31.0         16694.0        7.0
1       금호                  박성용          17.0         11606.0        8.0
13      한화                  김승연          25.0         11496.0        9.0
19      두산                  박용곤          18.0         11192.0       10.0
7       쌍용                  김석원          20.0          9039.0       11.0
11      한솔                  이인

['(주)대우건설', '대우조선해양(주)', '대한주택공사', '(주)문화방송', '지엠대우오토앤테크놀', '(주)케이티', '(주)케이티앤지', '하나로텔레콤(주)', '한국가스공사', '한국농촌공사', '한국도로공사', '한국전력공사', '한국철도공사', '한국토지공사', '현대오일뱅크(주)', '쌍용양회공업(주)', '하이닉스', '포스코']
   chaebol the_same_person_2006  n_affil_2006  ttl_asset_2006  rank_2006
16      삼성                  이건희          59.0        115924.0        1.0
51   현대자동차                  정몽구          40.0         62235.0        2.0
21    에스케이                  최태원          56.0         54808.0        3.0
24      엘지                  구본무          30.0         54432.0        4.0
13      롯데                  신격호          43.0         32961.0        5.0
27     지에스                  허창수          50.0         21827.0        6.0
45      한진                  조양호          22.0         20702.0        7.0
52   현대중공업                  정몽준           7.0         17267.0        8.0
46      한화                  김승연          31.0         16526.0        9.0
12      두산                  박용곤          18.0         13659.0       10.0
0

   chaebol the_same_person_2010  n_affil_2010  ttl_asset_2010  rank_2010
9       삼성                  이건희          67.0        192850.0        1.0
41   현대자동차                  정몽구          42.0        100775.0        2.0
14    에스케이                  최태원          75.0         87522.0        3.0
18      엘지                  구본무          53.0         78918.0        4.0
8       롯데                  신격호          60.0         67265.0        5.0
21     지에스                  허창수          69.0         43084.0        6.0
42   현대중공업                  정몽준          16.0         40189.0        7.0
0   금호아시아나                  박삼구          45.0         34942.0        8.0
34      한진                  조양호          37.0         30387.0        9.0
7       두산                  박용곤          29.0         26788.0       10.0
36      한화                  김승연          48.0         26391.0       11.0
15   에스티엑스                  강덕수          16.0         20901.0       12.0
17     엘에스                  구태회          44.0      

   chaebol the_same_person_2015  n_affil_2015  ttl_asset_2015  rank_2015
14      삼성                  이건희          67.0        351533.0        1.0
56   현대자동차                  정몽구          51.0        194093.0        2.0
22    에스케이                  최태원          82.0        152388.0        3.0
25      엘지                  구본무          63.0        105519.0        4.0
10      롯데                  신격호          80.0         93407.0        5.0
30     지에스                  허창수          79.0         58506.0        6.0
57   현대중공업                  정몽준          27.0         57472.0        7.0
50      한진                  조양호          46.0         38382.0        8.0
52      한화                  김승연          52.0         37954.0        9.0
9       두산                  박용곤          22.0         33073.0       10.0
19     신세계                  이명희          29.0         27010.0       11.0
20     씨제이                  이재현          65.0         24608.0       12.0
24     엘에스                  구태회          48.0      

   chaebol the_same_person_2019  n_affil_2019  ttl_asset_2019  rank_2019
18      삼성                  이재용          62.0        414547.0        1.0
53   현대자동차                  정몽구          53.0        223493.0        2.0
25    에스케이                  최태원         111.0        218013.0        3.0
29      엘지                  구광모          75.0        129616.0        4.0
15      롯데                  신동빈          95.0        115339.0        5.0
51      한화                  김승연          75.0         65636.0        6.0
35     지에스                  허창수          64.0         62913.0        7.0
54   현대중공업                  정몽준          31.0         54808.0        8.0
22     신세계                  이명희          40.0         36374.0        9.0
50      한진                  조원태          32.0         31730.0       10.0
23     씨제이                  이재현          75.0         31136.0       11.0
14      두산                  박정원          23.0         28456.0       12.0
17      부영                  이중근          24.0      

In [37]:
print(dictNonChaebol)

{'2001': ['(주)고합', '대우전자(주)', '포항제철', '하나로통신', '현대정유(주)'], '2002': ['포항제철', '하나로통신', '현대정유(주)', '한국통신', '농업기반공사', '담배인삼공사', '대한주택공사', '한국가스공사', '한국도로공사', '한국수자원공사', '한국전력공사', '한국토지공사'], '2003': ['농업기반공사', '대한주택공사', '(주)케이티', '(주)케이티앤지', '(주)포스코', '하나로통신(주)', '한국가스공사', '한국도로공사', '한국수자원공사', '한국전력공사', '한국토지공사', '대우자동차', '대우조선해양(주)', '(주)문화방송'], '2004': ['농업기반공사', '대우자동차', '대우조선해양', '대한주택공사', '(주)문화방송', '(주)케이티', '케이티앤지', '하나로통신(주)', '한국가스공사', '한국도로공사', '한국전력공사', '한국토지공사', '(주)대우건설', '지엠대우', '포스코'], '2005': ['농업기반공사', '(주)대우건설', '대우자동차', '대우조선해양', '대한주택공사', '(주)문화방송', '지엠대우', '(주)케이티', '케이티앤지', '하나로텔레콤(주)', '한국가스공사', '한국도로공사', '한국전력공사', '한국토지공사', '한국철도공사', '현대오일뱅크(주)', '포스코'], '2006': ['(주)대우건설', '대우조선해양(주)', '대한주택공사', '(주)문화방송', '지엠대우오토앤테크놀', '(주)케이티', '(주)케이티앤지', '하나로텔레콤(주)', '한국가스공사', '한국농촌공사', '한국도로공사', '한국전력공사', '한국철도공사', '한국토지공사', '현대오일뱅크(주)', '쌍용양회공업(주)', '하이닉스', '포스코'], '2007': ['대우조선해양(주)', '대한주택공사', '(주)문화방송', '쌍용양회공업(주)', '지엠대우', '(주)케이티', '(주)케이티앤지', '(주)포스코', '하나로텔레콤(주)', '(주)

In [38]:
# Save nonchaebol dictionary
with open('../data/overview/dictNonChaebol.pkl', 'wb') as f:
    pickle.dump(dictNonChaebol, f)

In [39]:
dfMrgd.head(50)

,chaebol,the_same_person_2001,n_affil_2001,ttl_asset_2001,rank_2001,the_same_person_2002,n_affil_2002,ttl_asset_2002,rank_2002,the_same_person_2003,...,ttl_asset_2020,rank_2020,the_same_person_2021,n_affil_2021,ttl_asset_2021,rank_2021,the_same_person_2022,n_affil_2022,ttl_asset_2022,rank_2022
0,삼성,이건희,64.0,69873.0,1.0,이건희,63.0,72351.0,1.0,이건희,...,424848.0,1.0,이재용,59.0,457305.0,1.0,이재용,60.0,483919.0,1.0
1,현대,정몽헌,26.0,53632.0,2.0,정몽헌,12.0,11784.0,7.0,정몽헌,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,엘지,구본무,43.0,51965.0,3.0,구본무,51.0,54484.0,2.0,구본무,...,136967.0,4.0,구광모,70.0,151322.0,4.0,구광모,73.0,167501.0,4.0
3,에스케이,최태원,54.0,47379.0,4.0,최태원,62.0,46754.0,3.0,최태원,...,225526.0,3.0,최태원,148.0,239530.0,3.0,최태원,186.0,291969.0,2.0
4,현대자동차,정몽구,16.0,36136.0,5.0,정몽구,25.0,41266.0,4.0,정몽구,...,234706.0,2.0,정의선,53.0,246084.0,2.0,정의선,57.0,257845.0,3.0
5,한진,조중훈,19.0,21307.0,6.0,조중훈,21.0,21596.0,5.0,조양호,...,33549.0,11.0,조원태,31.0,33600.0,11.0,조원태,33.0,35238.0,11.0
6,롯데,신격호,31.0,16694.0,7.0,신격호,32.0,17964.0,6.0,신격호,...,121524.0,5.0,신동빈,86.0,117781.0,5.0,신동빈,85.0,121589.0,5.0
7,금호,박성용,17.0,11606.0,8.0,박성용,15.0,10608.0,8.0,박성용,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,한화,김승연,25.0,11496.0,9.0,김승연,26.0,9892.0,10.0,김승연,...,71686.0,6.0,김승연,83.0,72898.0,6.0,김승연,91.0,80388.0,6.0
9,두산,박용곤,18.0,11192.0,10.0,박용곤,18.0,8988.0,11.0,박용곤,...,29251.0,12.0,박정원,22.0,29659.0,12.0,박정원,21.0,26334.0,13.0


In [40]:
dfMrgd['chaebol'].sort_values().unique()

array(['IMM인베스트먼트', 'KG', 'SM', '교보생명보험', '금호', '금호석유화학', '금호아시아나', '네이버',
       '넥슨', '넷마블', '농심', '다우키움', '대교', '대림', '대방건설', '대상', '대성', '대주건설',
       '대한전선', '대한해운', '동국제강', '동부', '동양', '동양화학', '동원', '두나무', '두산',
       '롯데', '메리츠금융', '미래에셋', '반도홀딩스', '보광', '보성', '부영', '삼보컴퓨터', '삼성',
       '삼양', '삼천리', '선명', '세아', '셀트리온', '신세계', '신영', '쌍용', '씨앤', '씨제이',
       '아모레퍼시픽', '아이에스지주', '애경', '에스케이', '에스티엑스', '에이치디씨', '엘에스', '엘지',
       '엘지전선', '엠디엠', '영풍', '오리온', '오씨아이', '오케이금융그룹', '웅진', '유진', '이랜드',
       '일진', '장금상선', '제일제당', '중앙', '중앙일보', '중흥건설', '지에스', '카카오', '케이씨씨',
       '코오롱', '크래프톤', '태광', '태광산업', '태영', '태평양', '프라임', '하림', '하이트맥주',
       '하이트진로', '한국타이어', '한국투자금융', '한라', '한솔', '한진', '한진중공업', '한화', '현대',
       '현대백화점', '현대산업개발', '현대자동차', '현대중공업', '현대해상화재보험', '호반건설', '효성'],
      dtype=object)

In [41]:
dfMrgd.tail(50)

,chaebol,the_same_person_2001,n_affil_2001,ttl_asset_2001,rank_2001,the_same_person_2002,n_affil_2002,ttl_asset_2002,rank_2002,the_same_person_2003,...,ttl_asset_2020,rank_2020,the_same_person_2021,n_affil_2021,ttl_asset_2021,rank_2021,the_same_person_2022,n_affil_2022,ttl_asset_2022,rank_2022
47,중앙일보,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,태평양,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,교보생명보험,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13580.0,21.0,신창재,12.0,14413.0,22.0,신창재,14.0,13817.0,27.0
50,오리온,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,웅진,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,태광,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8146.0,42.0,이호진,19.0,8767.0,41.0,이호진,19.0,9793.0,41.0
53,미래에셋,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,18554.0,16.0,박현주,38.0,19333.0,17.0,박현주,40.0,20231.0,18.0
54,유진,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5435.0,53.0,유경선,52.0,5528.0,53.0,유경선,57.0,5429.0,59.0
55,애경,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5631.0,51.0,장영신,37.0,5589.0,51.0,장영신,35.0,6198.0,53.0
56,한라,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7679.0,44.0,정몽원,15.0,8104.0,43.0,정몽원,15.0,9146.0,45.0


In [42]:
dfMrgd.to_excel("../data/overview/affilNumTtlAsset/affilNumTtlAsset2001-2022.xlsx", index=False)